In [6]:
# realised i should be using an autoencoder because my data isn't labelled

# node, edge, graph AEs

In [ ]:
# convert MLP to GNN by swapping torch.nn.linear with PyG's GNN operators e.g. GCN layer

In [32]:

import torch_geometric.nn as pyg_nn
import torch.nn.functional as F

from torch_geometric.data import DataLoader




In [15]:
from ts_vae.data_processors.grambow_processor import ReactionDataset
from torch_geometric.utils import train_test_split_edges
import torch.nn as nn
from torch_geometric.nn import GCNConv, GAE
import torch

In [16]:
base_path = r'data/'
r_dataset = ReactionDataset(base_path, geo_file = 'train_r') 

data = r_dataset.data
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data = data, val_ratio = 0, test_ratio = 0.2)

In [17]:
class LinearEncoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(LinearEncoder, self).__init__()
        # use single GC to get embeddings for nodes here
        self.conv = GCNConv(in_channels, out_channels)
    
    def forward(self, x, edge_index):
        # no relu for linearity
        return self.conv(x, edge_index)
    


In [19]:
num_node_fs = r_dataset.data.num_node_features # = 11
out_channels = 4

# build model and optimiser
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GAE(LinearEncoder(num_node_fs, out_channels))
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.01)


In [21]:
def train():
    model.train()
    opt.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    opt.step()
    return float(loss)

def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [23]:
epochs = 3
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 001, AUC: 0.7642, AP: 0.6247
Epoch: 002, AUC: 0.7648, AP: 0.6258
Epoch: 003, AUC: 0.7648, AP: 0.6262
